<a href="https://colab.research.google.com/github/basharbme/3D-Pelvic-Bone-cancer-segmentation-and-classification/blob/main/PBT_3D_Classification_Training_Part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install monailabel

     |████████████████████████████████| 6.2 MB 35.5 MB/s 
     |████████████████████████████████| 1.6 MB 45.7 MB/s 
     |████████████████████████████████| 74 kB 3.3 MB/s 
     |████████████████████████████████| 650 kB 43.3 MB/s 
     |████████████████████████████████| 51 kB 2.5 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 2.7 MB 42.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.9 MB 28.6 MB/s 
     |████████████████████████████████| 10.1 MB 41.5 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 636 kB 51.5 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 60 kB 7.7 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
     |████████████████████████████████| 316 kB 58.4 MB/s 
     |███████████

In [ ]:
!pip install monai-deploy-app-sdk

     |████████████████████████████████| 134 kB 32.0 MB/s 
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1


In [ ]:
!pip install SimpleITK==2.1.0
!pip install torchsummaryX
!pip install nibabel
!pip install pillow
!pip install tensorboard
!pip install gdown
!pip install pytorch-ignite==0.4.4
!pip install itk
!pip install tqdm
!pip install lmdb
!pip install psutil
!pip install pandas
!pip install einops
!pip install scikit-image
!pip install 'monai[all]'

     |████████████████████████████████| 48.4 MB 2.0 MB/s 
  Attempting uninstall: SimpleITK
    Found existing installation: SimpleITK 2.1.1
    Uninstalling SimpleITK-2.1.1:
      Successfully uninstalled SimpleITK-2.1.1
     |████████████████████████████████| 200 kB 35.1 MB/s 
  Attempting uninstall: pytorch-ignite
    Found existing installation: pytorch-ignite 0.4.5
    Uninstalling pytorch-ignite-0.4.5:
      Successfully uninstalled pytorch-ignite-0.4.5
     |████████████████████████████████| 3.1 MB 36.1 MB/s 
     |████████████████████████████████| 5.4 MB 34.8 MB/s 
  Using cached pytorch_ignite-0.4.5-py3-none-any.whl (221 kB)
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 895 kB 31.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
  Attempting uninstall: pytorch-ignite
    Found existing installation: pytorch-ignite 0.4.4
    Uninstalling pytorch-ignite-0.4.4:
      Successfully uninstalled pytorch-ignite-0.4.4


In [ ]:
import logging
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType


def main():
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

   
    data_path = os.sep.join(["/content/drive/MyDrive/tumours/"])
    images = [
        "1.nii",
        "2.nii",
         "3.nii",
         "4.nii",
         "5.nii",
         "6.nii",
          "7.nii",
          "8.nii",
           "9.nii",
           "10.nii",
           "11.nii",
           "12.nii",
           "13.nii",
           "14.nii",
           "17.nii",
           "18.nii",
           "19.nii",
           "21.nii",
           "22.nii",
           "23.nii",
           "24.nii",
           "25.nii",
           "26.nii",
    ]
    images = [os.sep.join([data_path, f]) for f in images]
     
    osteochondroma = 0
    osteoblastoma = 1
    primary_metastasis= 2
    primary_sarcoma= 3
    secondary_sarcoma= 4
     
    # 5 labels for tumour classification
    labels = np.array([ 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4], dtype=np.int64)

    # Define transforms
    train_transforms = Compose([ScaleIntensity(), AddChannel(), Resize((96, 96, 96)), RandRotate90(), EnsureType()])
    val_transforms = Compose([ScaleIntensity(), AddChannel(), Resize((96, 96, 96)), EnsureType()])

    # Define image dataset, data loader
    check_ds = ImageDataset(image_files=images, labels=labels, transform=train_transforms)
    check_loader = DataLoader(check_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())
    im, label = monai.utils.misc.first(check_loader)
    print(type(im), im.shape, label)

    # create a training data loader
    train_ds = ImageDataset(image_files=images[:22], labels=labels[:22], transform=train_transforms)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())

    # create a validation data loader
    val_ds = ImageDataset(image_files=images[-22:], labels=labels[-22:], transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())

    # Create DenseNet121, CrossEntropyLoss and Adam optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=5).to(device)
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 1e-5)

    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter()
    for epoch in range(5):
        print("-" * 22)
        print(f"epoch {epoch + 1}/{5}")
        model.train()
        epoch_loss = 0
        step = 0
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                num_correct = 0.0
                metric_count = 0
                for val_data in val_loader:
                    val_images, val_labels =val_data[0].to(device), val_data[1].to(device)
                    val_outputs = model(val_images)
                    value = torch.eq(val_outputs.argmax(dim=1), val_labels)
                    metric_count += len(value)
                    num_correct += value.sum().item()
                metric = num_correct / metric_count
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), "best_metric_model_classification3d_array_1.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current accuracy: {:.4f} best accuracy: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_accuracy", metric, epoch + 1)
    print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
    writer.close()


if __name__ == "__main__":
    main()

MONAI version: 0.7.0
Numpy version: 1.19.5
Pytorch version: 1.10.0+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: bfa054b9c3064628a21f4c35bbe3132964e91f43

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: 3.0.2
scikit-image version: 0.18.3
Pillow version: 7.1.2
Tensorboard version: 2.7.0
gdown version: 3.6.4
TorchVision version: 0.11.1+cu111
tqdm version: 4.62.3
lmdb version: 0.99
psutil version: 5.4.8
pandas version: 1.1.5
einops version: 0.3.2
transformers version: 4.12.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

<class 'torch.Tensor'> torch.Size([2, 1, 96, 96, 96]) tensor([0, 0])
----------------------
epoch 1/5
1/11, train_loss: 1.5586
2/11, train_loss: 1.9902
3/11, train_loss: 1.5438
4/11, train_loss: 1.4315
5/11, train_loss: 1.6034
6/11, train_loss: 1.5255
7/11, train_loss: 1.7641
8/11, train_loss: 1.4450
9/11, tr